In [1]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, classification_report, f1_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

data = pd.read_csv('Data/25day_dataset.csv')


C:\Users\danie\AppData\Local\Temp\ipykernel_9856\1430184262.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


Si etichetta il dataset con 4 classi:

    1) Tutti spenti [0]
    2) Solo lavatrice accesa [1]
    3) Solo lavastoviglie accesa [2]
    4) Solo forno acceso [3]

In [2]:
label = []
for i in range(len(data)):
    if data['wahing_machine'].iloc[i]==0 and data['dishwasher'].iloc[i]==0 and data['oven'].iloc[i]==0:
        label.append(0)
    elif data['wahing_machine'].iloc[i]>0:
        label.append(1)
    elif data['dishwasher'].iloc[i]>0:
        label.append(2)
    elif data['oven'].iloc[i]>0:
        label.append(3)
data['Class'] = label

In [3]:
#Droppa le colonne che non servono piu 
data.drop(['wahing_machine', 'dishwasher', 'oven'], axis=1, inplace=True)

di seguito le istruzioni fronite per effettuare il corretto folding:

Test set fold 01: 
Tutte le misure del giorno 2022-01-01


Test set fold 02: 
Tutte le misure del giorno 2022-01-02

Test set fold 03: 
Tutte le misure del giorno 2022-01-03

…..


Test set fold 09: 
Tutte le misure del giorno 2022-01-10
Tutte le misure del giorno 2022-01-11

Test set fold 10: 
Tutte le misure del giorno 2022-01-12
Tutte le misure del giorno 2022-01-13 (dati disponibili fino alle 03:16)


In [4]:
test_set_list=[]
Test_set1 =data[data['DateTime'].str.startswith('2022-01-01')]
Test_set2 =data[data['DateTime'].str.startswith('2022-01-02')]
Test_set3 =data[data['DateTime'].str.startswith('2022-01-03')]
Test_set4 =data[data['DateTime'].str.startswith('2022-01-04')]
Test_set5 =data[data['DateTime'].str.startswith('2022-01-05')]
Test_set6 =data[data['DateTime'].str.startswith('2022-01-06')]
Test_set7 =data[data['DateTime'].str.startswith('2022-01-07')]
Test_set8 =data[(data['DateTime'].str.startswith('2022-01-08')) | (data['DateTime'].str.startswith('2022-01-09'))]
Test_set9 =data[(data['DateTime'].str.startswith('2022-01-10')) | (data['DateTime'].str.startswith('2022-01-11'))]
Test_set10 =data[(data['DateTime'].str.startswith('2022-01-12')) | (data['DateTime'].str.startswith('2022-01-13'))]

In [5]:
test_set_list= [Test_set1, Test_set2, Test_set3, Test_set4, Test_set5, Test_set6, Test_set7, Test_set8, Test_set9, Test_set10]
Test_set=Test_set1.append([Test_set2,Test_set3,Test_set4,Test_set5,Test_set6,Test_set7,Test_set8,Test_set9,Test_set10])
Training_set= data.drop(Test_set.index, axis=0)

C:\Users\danie\AppData\Local\Temp\ipykernel_9856\1407526483.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Test_set=Test_set1.append([Test_set2,Test_set3,Test_set4,Test_set5,Test_set6,Test_set7,Test_set8,Test_set9,Test_set10])


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(Training_set.drop(['DateTime', 'Class'], axis=1), Training_set.Class, test_size=0.01, random_state= 8) 

print(len(X_train),len(X_val))

1026432 10368


In [8]:
#VALIDATION SVM OVR
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

svc = SVC()
parameters = {'kernel':('linear', 'rbf', 'poly', 'sigmoid'), 'C':[1, 10], 'decision_function_shape' : ['ovo','ovr']}
clf = GridSearchCV(svc, parameters)
clf.fit(X_val, y_val)
clf.best_params_

{'C': 10, 'decision_function_shape': 'ovo', 'kernel': 'rbf'}

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(Training_set.drop(['DateTime', 'Class'], axis=1), Training_set.Class, test_size= 0.01, random_state= 8) 

#imputation
from imblearn.under_sampling  import RandomUnderSampler
from collections import Counter
print(Counter(y_train), y_train.count())
n= 400000 - (y_train.values == 1).sum() - (y_train.values == 3).sum() - (y_train.values == 2).sum()
undersample = RandomUnderSampler(sampling_strategy={0: n-905, 1: (y_train.values == 1).sum(), 2: (y_train.values == 2).sum(), 3: (y_train.values == 3).sum()})
X_new, y_new = undersample.fit_resample(X_train, y_train)
print(Counter(y_new), y_new.count())


Counter({0: 967046, 1: 41769, 2: 13538, 3: 4079}) 1026432
Counter({0: 339709, 1: 41769, 2: 13538, 3: 4079}) 399095


In [8]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(X_train, y_train, test_size = 0.01, random_state= 8)
len(X_train)

1026432

In [9]:
#print(Counter(y_val1),y_val.count())
print(Counter(y_train),y_train.count())

undersample = RandomUnderSampler(sampling_strategy='auto')
X_new, y_new = undersample.fit_resample(X_train, y_train)
print(Counter(y_new), y_new.count())

Counter({0: 967046, 1: 41769, 2: 13538, 3: 4079}) 1026432
Counter({0: 4079, 1: 4079, 2: 4079, 3: 4079}) 16316


In [33]:
#VALIDATION SVM OVR
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

svc = SVC(kernel='rbf', decision_function_shape='ovo')
parameters = {'C':[1, 10, 40, 30, 20]}
clf = GridSearchCV(svc, parameters)
clf.fit(X_val, y_val)
clf.best_params_

{'C': 40}

In [ ]:
https://stats.stackexchange.com/questions/437072/use-f1-score-in-gridsearchcv

In [11]:
from sklearn.svm import SVC

model = SVC(C=40, kernel='rbf', decision_function_shape='ovo')
model.fit(X_new, y_new)
y_preds=model.predict(Test_set.drop(['DateTime', 'Class'], axis=1))
print("CM:\n" + str(confusion_matrix(Test_set.Class,y_preds)) + "\n")
print(classification_report(Test_set.Class,y_preds))

CM:
[[836009 152566  62153   7239]
 [ 15474  19640    333    285]
 [  1985   3274   9281    111]
 [  2245   2572    291   9742]]

              precision    recall  f1-score   support

           0       0.98      0.79      0.87   1057967
           1       0.11      0.55      0.18     35732
           2       0.13      0.63      0.21     14651
           3       0.56      0.66      0.60     14850

    accuracy                           0.78   1123200
   macro avg       0.44      0.66      0.47   1123200
weighted avg       0.93      0.78      0.84   1123200



In [12]:
model = SVC(C=40, kernel='rbf', decision_function_shape='ovr')
model.fit(X_new, y_new)
y_preds=model.predict(Test_set.drop(['DateTime', 'Class'], axis=1))
print("CM:\n" + str(confusion_matrix(Test_set.Class,y_preds)) + "\n")
print(classification_report(Test_set.Class,y_preds))

CM:
[[836009 152566  62153   7239]
 [ 15474  19640    333    285]
 [  1985   3274   9281    111]
 [  2245   2572    291   9742]]

              precision    recall  f1-score   support

           0       0.98      0.79      0.87   1057967
           1       0.11      0.55      0.18     35732
           2       0.13      0.63      0.21     14651
           3       0.56      0.66      0.60     14850

    accuracy                           0.78   1123200
   macro avg       0.44      0.66      0.47   1123200
weighted avg       0.93      0.78      0.84   1123200



In [35]:
Test_set=test_set_list[2]
y_preds=model.predict(Test_set.drop(['DateTime', 'Class'], axis=1))
print("CM:\n" + str(confusion_matrix(Test_set.Class,y_preds)) + "\n")
print(classification_report(Test_set.Class,y_preds))

CM:
[[26439 49549  5102     5]
 [    0  1022     2     0]
 [   42  1107  2365    63]
 [    0     1     0   703]]

              precision    recall  f1-score   support

           0       1.00      0.33      0.49     81095
           1       0.02      1.00      0.04      1024
           2       0.32      0.66      0.43      3577
           3       0.91      1.00      0.95       704

    accuracy                           0.35     86400
   macro avg       0.56      0.75      0.48     86400
weighted avg       0.96      0.35      0.49     86400



In [25]:
model = SVC(C=15, kernel='rbf', decision_function_shape='ovr')
model.fit(X_new, y_new)
y_preds=model.predict(Test_set.drop(['DateTime', 'Class'], axis=1))
print("CM:\n" + str(confusion_matrix(Test_set.Class,y_preds)) + "\n")
print(classification_report(Test_set.Class,y_preds))

CM:
[[787464 148341  82257  39905]
 [ 16410  14162   2678   2482]
 [  3141   2179   9186    145]
 [  1639   2375    840   9996]]

              precision    recall  f1-score   support

           0       0.97      0.74      0.84   1057967
           1       0.08      0.40      0.14     35732
           2       0.10      0.63      0.17     14651
           3       0.19      0.67      0.30     14850

    accuracy                           0.73   1123200
   macro avg       0.34      0.61      0.36   1123200
weighted avg       0.92      0.73      0.81   1123200



In [26]:
model = SVC(C=15, kernel='poly', decision_function_shape='ovr')
model.fit(X_new, y_new)
y_preds=model.predict(Test_set.drop(['DateTime', 'Class'], axis=1))
print("CM:\n" + str(confusion_matrix(Test_set.Class,y_preds)) + "\n")
print(classification_report(Test_set.Class,y_preds))

CM:
[[1017369   12720    4684   23194]
 [  24740    1065    4661    5266]
 [   4940     731    8084     896]
 [   3095     964     796    9995]]

              precision    recall  f1-score   support

           0       0.97      0.96      0.97   1057967
           1       0.07      0.03      0.04     35732
           2       0.44      0.55      0.49     14651
           3       0.25      0.67      0.37     14850

    accuracy                           0.92   1123200
   macro avg       0.43      0.55      0.47   1123200
weighted avg       0.92      0.92      0.92   1123200



In [ ]:
#VALIDATION SVM OVR
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

svc = SVC()
parameters = {'C':[1, 10, 20, 30, 20]}
clf = GridSearchCV(svc, parameters)
clf.fit(X_val, y_val)
clf.best_params_

In [14]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(X_train, y_train)
y_preds=neigh.predict(Test_set.drop(['DateTime', 'Class'], axis=1))
print("CM:\n" + str(confusion_matrix(Test_set.Class,y_preds)) + "\n")
print(classification_report(Test_set.Class,y_preds))


C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


CM:
[[978802  64053   5197   9915]
 [ 21350  14265     50     67]
 [  4385   3249   6984     33]
 [  7322   4770      5   2753]]

              precision    recall  f1-score   support

           0       0.97      0.93      0.95   1057967
           1       0.17      0.40      0.23     35732
           2       0.57      0.48      0.52     14651
           3       0.22      0.19      0.20     14850

    accuracy                           0.89   1123200
   macro avg       0.48      0.50      0.47   1123200
weighted avg       0.93      0.89      0.91   1123200



In [16]:
#knn con imputation
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(X_new, y_new)
y_preds=neigh.predict(Test_set.drop(['DateTime', 'Class'], axis=1))
print("CM:\n" + str(confusion_matrix(Test_set.Class,y_preds)) + "\n")
print(classification_report(Test_set.Class,y_preds))


C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


CM:
[[921571 115623   8516  12257]
 [ 18524  17037     81     90]
 [  3860   3688   7057     46]
 [  6515   5279     39   3017]]

              precision    recall  f1-score   support

           0       0.97      0.87      0.92   1057967
           1       0.12      0.48      0.19     35732
           2       0.45      0.48      0.47     14651
           3       0.20      0.20      0.20     14850

    accuracy                           0.84   1123200
   macro avg       0.43      0.51      0.44   1123200
weighted avg       0.93      0.84      0.88   1123200



In [19]:
len(X_val)

10368

In [20]:
#VALIDATION SVM OVR
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

svc = SVC(decision_function_shape = 'ovo')
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
clf = GridSearchCV(svc, parameters)
clf.fit(X_val, y_val)
clf.best_params_


{'C': 10, 'kernel': 'rbf'}

In [21]:
model = SVC(C=10, kernel='rbf', decision_function_shape='ovo')
model.fit(X_train, y_train)
y_preds=neigh.predict(Test_set.drop(['DateTime', 'Class'], axis=1))
print("CM:\n" + str(confusion_matrix(Test_set.Class,y_preds)) + "\n")
print(classification_report(Test_set.Class,y_preds))

In [20]:
# validation
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [100, 300, 500],
    'min_samples_leaf': [50, 100, 150],
    'criterion' : ['gini', 'entropy', 'log_loss']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_val, y_val)
CV_rfc.best_params_

{'criterion': 'entropy', 'min_samples_leaf': 50, 'n_estimators': 100}

In [9]:
#1° addestramento
from sklearn.ensemble import RandomForestClassifier
#Test_set= test_set_list[2]
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', min_samples_leaf=50)
clf.fit(X_train, y_train)
y_preds= clf.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,y_preds)) + "\n")
print(classification_report(Test_set.Class,y_preds))

CM:
[[1053198    3817     579     373]
 [   2874   32858       0       0]
 [   5148     594    8909       0]
 [   6154     804       1    7891]]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99   1057967
           1       0.86      0.92      0.89     35732
           2       0.94      0.61      0.74     14651
           3       0.95      0.53      0.68     14850

    accuracy                           0.98   1123200
   macro avg       0.94      0.76      0.83   1123200
weighted avg       0.98      0.98      0.98   1123200



In [11]:
#2° addestramento
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB


model =GaussianNB()
ovr_g = OneVsRestClassifier(model)
ovr_g.fit(X_train, y_train)
predicted = ovo_r.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,predicted)) + "\n")
print(classification_report(Test_set.Class,predicted))

CM:
[[1007868   25973   10260   13866]
 [   8745   25517     595     875]
 [   3671     529    8151    2300]
 [   4051    4255     456    6088]]

              precision    recall  f1-score   support

           0       0.98      0.95      0.97   1057967
           1       0.45      0.71      0.55     35732
           2       0.42      0.56      0.48     14651
           3       0.26      0.41      0.32     14850

    accuracy                           0.93   1123200
   macro avg       0.53      0.66      0.58   1123200
weighted avg       0.95      0.93      0.94   1123200



In [13]:
#2° addestramento
from sklearn.multiclass import OneVsOneClassifier
from sklearn.naive_bayes import GaussianNB


model =GaussianNB()
ovo_g = OneVsOneClassifier(model)
ovo_g.fit(X_train, y_train)
predicted = ovo_g.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,predicted)) + "\n")
print(classification_report(Test_set.Class,predicted))

CM:
[[1004991   14300   11758   26918]
 [   7816   22951     423    4542]
 [   3557     546    9357    1191]
 [   4051    1074     201    9524]]

              precision    recall  f1-score   support

           0       0.98      0.95      0.97   1057967
           1       0.59      0.64      0.62     35732
           2       0.43      0.64      0.51     14651
           3       0.23      0.64      0.33     14850

    accuracy                           0.93   1123200
   macro avg       0.56      0.72      0.61   1123200
weighted avg       0.96      0.93      0.94   1123200



In [12]:
#3° addestramento
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC


model = SVC(max_iter=1000)
ovr = OneVsRestClassifier(model)
ovr.fit(X_train, y_train)
predicted = ovr.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,predicted)) + "\n")
print(classification_report(Test_set.Class,predicted))

C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundatio

CM:
[[1049559     152       0    8256]
 [  27828    6226       0    1678]
 [   5126    9415       0     110]
 [   8294       1       0    6555]]



C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      0.99      0.98   1057967
           1       0.39      0.17      0.24     35732
           2       0.00      0.00      0.00     14651
           3       0.39      0.44      0.42     14850

    accuracy                           0.95   1123200
   macro avg       0.44      0.40      0.41   1123200
weighted avg       0.92      0.95      0.93   1123200



C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ensamble classificatori

In [21]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
X_train, X_t, y_train, y_t = train_test_split(Test_set.drop(['DateTime', 'Class'], axis=1), Test_set.Class, test_size=0.01, random_state= 8) 


eclf = VotingClassifier(estimators=[('ovr', ovr), ('rf', clf), ('gnb', ovo_g)],voting='hard')

eclf.fit(X_t,y_t)
predicted = eclf.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,predicted)) + "\n")
print(classification_report(Test_set.Class,predicted))

'''
for clf, label in zip([ovr, clf, ovo_g, eclf], ['one_VS_Rest', 'Random Forest', 'naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X_t, y_t, scoring='accuracy', cv=5)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
'''

C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundatio

CM:
[[1055862    1401      33     671]
 [   9001   26728       0       3]
 [   5176     547    8928       0]
 [   4949       4      37    9860]]

              precision    recall  f1-score   support

           0       0.98      1.00      0.99   1057967
           1       0.93      0.75      0.83     35732
           2       0.99      0.61      0.76     14651
           3       0.94      0.66      0.78     14850

    accuracy                           0.98   1123200
   macro avg       0.96      0.75      0.84   1123200
weighted avg       0.98      0.98      0.98   1123200



'\nfor clf, label in zip([ovr, clf, ovo_g, eclf], [\'one_VS_Rest\', \'Random Forest\', \'naive Bayes\', \'Ensemble\']):\n    scores = cross_val_score(clf, X_t, y_t, scoring=\'accuracy\', cv=5)\n    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))\n'

In [18]:
X_train, X_t, y_train, y_t = train_test_split(Test_set.drop(['DateTime', 'Class'], axis=1), Test_set.Class, test_size=0.01, random_state= 8) 
print(len(X_train), len(X_t))

1111968 11232


Per esportare il modello utilizziamo pickle

In [ ]:
import pickle
